In [34]:
import pandas as pd
import numpy as np

In [35]:
df = pd.read_csv('spotify_millsongdata.csv')
df.head(3)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...


In [36]:

df = df.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [37]:
df['text'] = df['text'].str.lower().replace(r'[^\w\s]','').replace(r'\n',' ', regex=True)

In [38]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [39]:
df['text'] = df['text'].apply(lambda x: tokenization(x))



In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [42]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [43]:
similarity[0]

array([1.        , 0.05354204, 0.00617834, ..., 0.0056004 , 0.00537423,
       0.02383589])

In [44]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [51]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs
    


In [52]:
recommendation('Crying Over You')


IndexError: index 0 is out of bounds for axis 0 with size 0

In [47]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))

In [48]:
recommendation('Dancing Queen')

'Song not found in the DataFrame. Please check the song name.'